# How to create a Laplacian pyramid

## Compiler parameters

Set my Jypyter environment for the use of OpenCV in a C++ notebook. You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use CMake.

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
#include <vector>
#include <sstream>
#include <string>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Create a structure

to store the pyramid.

In [5]:
vector<Mat> pyramid;

## 1. Read the image from a file on the disk and return a new matrix

![Original image](../level0.jpg)

In [6]:
Mat original_image = imread("../level0.jpg", IMREAD_COLOR);
original_image.convertTo(original_image, CV_32FC1);

## 2. Make sure its size is a power of two

This is image is 

In [7]:
cout << original_image.cols << "x" << original_image.rows << " pixels" << endl;

1024x695 pixels


- 1024 is a  power of two.
- 695 is not.

## 3. I'll mirror it to make it a power of two.

In [8]:
Mat non_blurred;
copyMakeBorder( original_image, non_blurred, 0, 1024 - original_image.rows, 0, 0, BORDER_REFLECT);

## 4. Blur the image

In [9]:
Mat blurred;
GaussianBlur(non_blurred, blurred, Size(3, 3), 0);

## 5. Pixelwise subtraction

In [10]:
Mat level1 = non_blurred - blurred;

## 6. Store this image in the pyramid

In [11]:
pyramid.push_back(level1);

Mat temp;
cv::normalize(pyramid.back(), temp, 0, 1, NORM_MINMAX, CV_32FC3);
imwrite("../level1-laplacian.png", 255 * temp);

![Level1](../level1-laplacian.png)

## 7. Reduce its size by half

(downsampling, i.e. no interpolation)

In [12]:
resize(blurred, non_blurred, Size(), 0.5, 0.5, INTER_NEAREST);

## 8. Iterate

Blur the image

In [13]:
GaussianBlur(non_blurred, blurred, Size(3, 3), 0);

Pixelwise subtraction

In [14]:
Mat level2 = non_blurred - blurred;

Store this image in the pyramid

In [15]:
pyramid.push_back(level2);

cv::normalize(pyramid.back(), temp, 0, 1, NORM_MINMAX, CV_32FC3);
imwrite("../level2-laplacian.png", 255 * temp);

Reduce its size by half (downsampling, i.e. no interpolation)

In [16]:
resize(blurred, non_blurred, Size(), 0.5, 0.5, INTER_NEAREST);

![Level 2](../level2-laplacian.png)

## 9. Iterate one more time

In [17]:
GaussianBlur(non_blurred, blurred, Size(3, 3), 0);

In [18]:
Mat level3 = non_blurred - blurred;

In [19]:
pyramid.push_back(level3);

cv::normalize(pyramid.back(), temp, 0, 1, NORM_MINMAX, CV_32FC3);
imwrite("../level3-laplacian.png", 255 * temp);

In [20]:
resize(blurred, non_blurred, Size(), 0.5, 0.5, INTER_NEAREST);

## 10. I got enough, stop here

In [21]:
pyramid.push_back(non_blurred);

cv::normalize(pyramid.back(), temp, 0, 1, NORM_MINMAX, CV_32FC3);
imwrite("../level4-laplacian.png", 255 * temp);

![Level 4](../level4-laplacian.png)

## All the levels

### Level 1
![Level 1](../level1-laplacian.png)

### Level 2
![Level 2](../level2-laplacian.png)

### Level 3
![Level 3](../level3-laplacian.png)

### Level 4
![Level 4](../level4-laplacian.png)

## Upsample all the images

For comparing the levels

In [22]:
for (int i = 1; i < pyramid.size(); i++)
{
    Mat temp = pyramid[i];
    resize(temp, temp, Size(pyramid[0].cols, pyramid[0].rows), 0, 0, INTER_NEAREST);
    
    stringstream file_name;

    cv::normalize(temp, temp, 0, 1, NORM_MINMAX, CV_32FC3);

    file_name << "../level" << i + 1 << "_upscaled-laplacian.png";
    imwrite(file_name.str(), 255 * temp);
}